# Overview

## Properties of remote computing

Resources such as compute and storage can be accessed anywhere, virtually unlimited, data is permanent (resilient against crashes), easily shareable, many different types of compute resources

## Characteristics of Cloud based Computing

On-demand resources (pay as you go)   
Resource Scalability (increased resources allocated to you as you need more)  
API-based access

# Virtual Machines

Virtual Machine is a process which is an OS running on existing system's OS.  
The existing system's OS is referred to as the **hypervisor**. 
VMs are useful because:
1. Easy way to use different OS
2. Fully utilize node capacity
3. Isolation- run shitty code on separate VMs

Key aspects are Host (physical machine), guest (VM on some OS), VMM/Hypervisor (software layer performing virtualization of the host)

## Virtualization

Suite of techniques that enable multiple Guests each running a separate OS to be run on the same physical machine.   
Challenges: 
1. Isolate guests from each other
2. Support different OS
3. Keep performance overhead small  
  
Full Virtualization - exact copy of the host hardware exposed to guest OS - no change required to guest OS, hard for x86    
Paravirtualization - similar to full virtualization, but not the exect copy of the host hardware is exposed, requires modification to Guest OS but no modification to guest applications

## XEN

Hypervisor that uses paravirtualization technique, requires modification of guest OS.   
XEN runs as the most priveleged software (ring 0), whereas guest OS runs at a lower privelege level (ring 1)

### XEN and syscalls

Naive setup: Guest OS calls XEN to execute each exception, XEN calls processor to execute syscall, processor executes syscall and returns control to XEN which returns control to OS.  
**This is inefficient as fuck**  
Faster exception handlers: Guest OS calls XEN to register an exception handler table. XEN validates the table, and places it in memory where the processor can find it. The guest OS executes a system call, and the processor directly accesses the system call handler table without XEN, and returns back to XEN after completion.   
**The idea here is we removed a level of abstraction, XEN, by storing exception handler table into memory, which made syscalls/exceptions much faster**

### XEN and Page Faults

What is a page fault? When the CPU tries to access a page not mapped in the virtual address space of a process 
Normal demand paging:  
Page table used to translate virtual addresses to physical addresses. Each process needs its own table, and the OS setsup the page table for a process when it starts the process.

### Normal Page fault handling

1. MMU notices that a particular virtual address cannot be translated to a physical address, and then notifies the CPU about this.
2. CPU sends an interrupt to the OS called the page fault
3. OS page fault handler is at a fixed memory location, and control transferred to this page fault handler.  
4. OS uses a page replacement algorithm to determine whether or not a page will be replaced and written to disk or not.
5. It reads the new code into that newly emptied page frame, and updates the page table entry. It then re-executes the faulting instruction, which should work.  
  
CPU works with virtual addresses, MMU contains page table per process which is used to translate virtual addresses to physical addresses, and this page table is set up by the OS. When a page fault occurs, the OS helps resolve it.

### XEN page fault handling

1. Guest OS registers page table with XEN
2. Processor encounters page fault, and transfers control to XEN (hardware exception table set to do this at boot time)
3. XEN reads the faulting page address from the processor register and saves it in the execution stack frame of the guest OS, and resolves it
4. XEN restarts the faulting instruction from the execution stack frame of the guest OS   
  
CPU works with virtual addresses, Guest OS registers per process page table with XEN, XEN sets up the page table inside MMU when it schedules a guest OS. MMU contains page table for a process executing within the Guest OS. At boot time, XEN sets the page fault handler to transfer control to itself. WHen the CPU encounters a page fault, control is transferred to XEN and XEN resolves the page fault

### Virtualizing I/O

XEN manages I/O devices, and the guest OS calls XEN for I/O.   
XEN uses shared memory regions between itself and the guest OS as a fast data transfer mechanism, and there's an asynchronous event mechanism much like DMA to notify the guest OS when I/O is complete

### Terms and Concepts

Domains: Guest OS/VMs are called Domains.  
Domain0: priveleged comain created by XEN on startup, control center for creating and managing other domains

### VMs in the Cloud

Must support different hardwars and supervisors, and support the provisioning of Virtual Machines remotely (i.e. allow API access to spin up VMs)

### Libvirt

Node/Host: Physical Machine   
Hypervisor: Layer of software that virtualizes a node in a set of virtual machines    
Domain: An instance of an OS running on a Node   
Goal: provide a common and stable layer sufficient to securely manage domains on a node.   
  
Libvirt daemon (background process - remember you can run things in the background with &)   
All interactions go through this daemon, and local communication happens over Unix sockets. Many available methods, and various Domains (Guest OS) defined using XML

# EC2

Instance Type- determines the hardware of the host computer used for launching your instances.  
Instances- Running VM.    
AMI- OS definition, multiple instatnces can be launched from the same AMI  
Goal of instance types is to support different kinds of applications.   
Perks: General purpose, compute and memory optimized, accelerated computing, storage optimized

### AMI - Amazon Machine Image

OS image that should be used for booting VM.  
Information included is hardware architecture on which AMI is built, virtualization type for which the AMI is built, where to find root volume, which contains instructions on how to boot the AMI

### EC2 run_instances

1. Find out a hardware node corresponding to the provided instance type and has enough capacity
2. Fetch specified AMI
3. Create volumes on the Elastic Block Storage corresponding to the block device mapping
4. Invoke the libvirt driver on the selected Node to start a domain using the AMI fetched

### EC2 Virtualization

Paravirtual, and HVM (Hardware virtual machine)- uses special virtualization extensions, allows running guest OS without any modifications

### EC2 Instance initialization

Initializing an instance at startup contains user data and is done via cloud-init. User data is used when initialization is done using bash scripts. Logs stored in /var/log/cloud-init-output.log.   
Various use cases including an instance with a specific applicatino running, enabling remote logins, injecting app specific config files

### Security Groups

Security groups act as a virtual firewall, you can choose to allow traffic on specific ports. When you launch an instqance, you can associate one or more security groups with an instance. Rules need to hbe added to the security groups that control traffic to/from the instance.   
By default, all outbound traffic is allowed, you can allow certain protocols, port ranges, source/destinations, and all traffic (0.0.0.0/0)   
Each account gets a default security group per VPC (Virtual private cloud, abstraction for managing set of AWS resources as a unit. During instance creation is is inside the default VPC).

### CIDR

IP Address: 32 bit, divided into Network number and host number.   
CIDR Addressing: 
IP and Mask, mask is used to obtain network number. Indicates how many most significant bits should be treated as 1 while filtering.  
Written as IP Address/Mask: 1272.16.0.0/16: Mask is 16 bits in size, 16 most significant bits should be treated as 1.   
Mask is used in routing table to determine how to route a packet, used in security group rules to detemine wheter to accept a packet

### SSH Key Pair

Use an SSH key pair to connect to the EC2 instance, Public key stored by Amazon, private key stored by you.  
When the user logs in:
1. EC2 instance is authenticated by verifying the host key
2. User SSH client creates a unique message and encrypts it with the User's private key
3. EC2 instance uses User's public key to verify the message by decrypting it

# IAM

## Authentication

System generates identity credentials for you, you present them to the system whenever required

## Authorization

What are you allowed to do?  
System defines permissions corresponding to your identity. System encodes permissions in Authorization Policies, when you invoke any action, the system verifies that your identity is allowed to perform the action according to Authorization Policies.   
Authorization checking comes after authentication is successful

## Unix Identities

Users: individuals, processes  
Group: mechanism to define permissions for a set of users. Effective permissions that a user gets is union of the user's individual permissions and all the permissions for the grups that the user is part of. Users are added to a group and they remain in that group forever.

## DB Systems / Workflow Systems

Access control decisions based on the functions a user is allowed to perform within an organization.   
Mechanism to introduce dynamism into User's permissions. Users are added/removed from a Role by the sysadmin. When a user is in a role, the effective permissions of the User are permissions available in that Role

## AWS Authentication and Authorization

Authentication done using:
1. Username and password for the web console
2. Access key for programmatic access, stored in ~/.aws/credentials

## Request Signing

Verifying that the HTTP request came from authenticated user, contains a MAC (Message Authentication code), shared secret key with the Access Key and ID

## IAM Identities

AWS Account root user - Email address and password used to sign up with AWS, use root user to cerate other IAM Users.  
IAM Users- represents person or service that interacts with AWS (name and password to sign into AWS Console, Access Keys for programmatic access)   
IAM Groups- collection of IAM users.   
Roles- permissions for IAM users

IAM Users are part of an AWS account, contain a resource URN (called ARN) in the following format: arn:aws:iam::<account-ID-without-hyphens>:user/<username>.  
Credentials contain access keys, passswords and username for console. Default permissions is nothing, but can be individually assigned by root user

Service Account - IAM user that is created solely for programmatic access is called a service account, two keys allowed at a time, and key rotation exists for creating new keys

IAM Group is a collection of users that allows you to specify permissions for multiple users, making it easier to manager permissions for those users. A group can contain many users and a user can belong to multiple groups. Groups can't be nested, can only contain users.

## Permissions and Policy

Permission is a action resouce pair. A policy is a dictates a when/who/where/how a permission is to be given out.  
Principal based Policies- policies that are attached to a user     
Resource based policies- policies attached to a resource

## Policy Elements

1. Effect: Allow/Deny
2. Action: Service Name: Action Name pair. Case insensitive, and each AWS service specifies the actions it supports. Multiple actions is supported, and you can use * to represent all actions of a service.
3. Resource: Specifies the object of the polocy, specified using Amazon Resource Names, use * to indicate any resource.

## Policy Variables

You can use policy variables to reuse policy definitions rather than having to create 1000 of the same policy.  
Define a policy with policy variables and attach it to the user.   
When a request comes in, AWS will check what policy variables are defined in the policy and pass them to the policy enforcement engine, which will check values against the policy

## Use Cases

User logging into the AWS console, SDK/CLI into AWS, service to service authorization, mobile app to AWS service, Cross-account services, access control issues

## IAM Role

Set of permissions with an action resource pair.   
Who can assume a role?
1. IAM user with the same AWS account as the role
2. IAM user with the different AWS account as the role
3. Web service offered by AWS
4. External user authenticated using external identity provider.  
  
Role does not have password or access keys, when a user assumes a role, temporary credentials are created and provided to the user   
Contains a permission policy (has a set of permissions for Role), and a trust policy (who can assume the role), permissions to pass the role to someone else.   


## IAM Roles and EC2

Special role for EC2 instance, given to each instance. Role defines authorization policies for any application running on the EC2 instance. Application can obtain temporary credentials by querying instance metadata service from instance.  
Instance metadata corresponding to a launched instance, accessible inside running instance i.e. curl http://169.254.169.254/latest/meta-data/

## Web identity Federation

Use it to authenticate application using identity provider, get a security token after authentication is done, use token to make calls to S3, typically uses OAuth 2.0

# S3

### Buckets

Container where objects are stored, region specific (objects stored in region never leave region unless explicitly transferred), bucket is owned by AWS account that created it

### Objects

Objects are things that are stored in buckets, has metadata, gets a unique key and version ID, and is identified by that same kay and version ID

### Data Storage

Data is replicated across multiple servers within Amazon's data centers. 
Consistent hashing is used to determine which servers should hold each item.  

### Replication

S3 stores each object across multiple facilities before returning success. A facility is an availabiliy zone within a region. We don't actually know how many facilities exist, but S3 claims to sustain loss of data in two facilities, so every object is stored in at least three facilities.  
They guarantee 99.99% availability, which means that the downtime allowed is 52 minutes and 33 seconds

### Cross Region replication

Asynchronous copying of objects across buckets between regions, source and destination buckets can be owned by different accounts.

### Data Consistency

Replication has issues, can affect the consistency of the data in the system, which is defined as how a data object will be handled by a system. 
1. When a new object is inserted and queried back, does the system return this object immediately?
2. When an existing object is modified and queried back, does the system return the modified object immediately?
3. When an existing object is deleted and queried, what does the system do?   
  
Strong consistency- after an update, any access will return the updated value.   
Eventual consistency- if no new updates are made to an object, then eventually all accesses will return the last updated values

### S3 Consistency Model

Uses Eventual Consistency - changes to S3 object state is eventually reflected to all nodes.  
Insert Object - Read after write consistency for retrieving the object itself, eventually consistent for listing keys in bucket (may not show object immediately). 
Update Object - Eventually consistent, updates to an object may not reflect immediately.  
Delete Object - eventually consistent, deleted object may still show up in listing bucket, and all replicas

### Object handling

S3 does not support object locking, if two PUT requests are simultaneously made, the request with the latest timestamp wins. Object locking needs to be programmed into the application.   
No way to make atomic updates. Cannot make update of one key dependent on the update of another key, needs to be programmed in application

### Buckets (again)

100 buckets per account, names need to be unique across all of AWS, no limit on objects storable in bucket, bucket cannot be created inside another bucket   
Bucket names must be between 3 to 63 characters, and must be a series of one or more labels separated by periods and must start with lower case letters or a number. They cannot be formatted as IP addresses

### Objects

1. Key: Name you assign to object used to retrieve it
2. Version ID: generated by S3 when added to bucket
3. Value: Content of object
4. Metadata: Set of name value pairs associated with object
5. Subresources: object specific addition information
6. Access control information- Object/bucket/user access control policy.  
  
Buckets and objects private by default, explicitly need to grant access if we want other to access an object.  
Metadata for objects can be set only when uploading the object.  
If modification is required, need to make a copy of the object and modify the object.   
System defined metadata - metadata generated by the system   
User defined metadata - metadata provided by the user.   
Object names can be used to simulate hierarchical organization, object names include any valid UTF-8 Character   
Objects can be tagged, similar to metadata, but can be added after creation.

### Object Versioning

Versioning needs to be enabled at a bucket level, each object will get a unique version ID once versioning is enabled. Each version of an object is a complete Object (you are charged for each version separately). GET object will return latest version if no version ID is specified. Individual versions need to be deleted specifically   
Version ID is a part of the Metadata, but Metadata can't be changed. This is circumvented by the fact that new objects are created for each version created.

### Access Management

Permissions can be granted to IAM Users, accounts, authenticated users, everyone. 
Access control list defined permissions, can be attached to bucket or object.   
Grantee: AWS account/S3 groups that have access, cannot be an IAM user   
Canned ACL: Predefined ACLs provided by AWS, specifiy an ACL using an HTTP request header

### Bucket operation request authorization

Convert all permissions into a set of policies to evaluate at run time, evaluates user and then bucket context, which is an identification of what policies should be evaluated and under what authority, and authority is an entity tht should be used to resolving any policy variables

### Object Operation request authorization

Convert all relevant permissions into a set of policies to evaluate at run time, evaluated in user, bucket, object context order.

### Using S3 to host static sites

You can use S3 to host static sites that don't allow user input. This can be configured at bucket level

# Cloud Object Storage

Object storage is a service that allows storing different types of data files. Each data item is an object, which gets a unique identifier. Data items are accessed using these identifiers

### Object Storage Service

1. Should be scalable (store large amounts of data), handle big files and large numbers of concurrent clients
2. Should be fast in performing operationr
3. An object should be available until the user deletes it
4. It should be possible to control access to each object.
5. Should be possible to define structure when storing objects

### Design Requirements

Store each object as a file on a node, use more than one node for storage, replicate data across nodes.  
Support simple put/get operations, object transfer, and object storage mechanism in addition to access control policies

### Design constraints

Nodes can change over time, each object/file should have a primary node, other nodes that store the object sare replicas, and client requests can be handled by either the primary or replica nodes

### Node Dynamism

How to store data when number of nodes are constant? Data should be distributed evenly across nodes, but how do you store data when number of nodes is not constant?

Need a data structure that has fast insertion, uniform distribution. Hash table is good, but table can become full. Rehashing is not an option, as it requires moving items from node to node.

### Consistent Hashing

Instead of statically assigning a slot number to a node, the assignment is made through the hash function. Maps the nodes and the keys to the same range in the hash space, and each node is in charge of keys whose hash values are less than or equal to the node's own hash value, but greather than the hash value of the previous node.

### How to efficiently find required Node?

Organize nodes in the form of a ring based on the hashing. Maintain an inorder successor for each node.   
O(Tree height) for insertion, O(logN) for search

# Eventual Consistency

Service should be available as much as possible, and be consistent (i.e. if it's supposed to be replicated 3 times, it should maintain 3 copies).

### Non partitioned Setting

All nodes reachable, maintaining data consistency is not a problem. Service and store data in any node, service can read data item from every node as well.

### Partitioned Setting

What if service denies client PUT request for data item since partitions are broken, or if data on other partitions is inconsistent with newest data.   
Replica nodes can end up on different partitions.

If the service wants to remain consistent, it may need to deny certain requests until partitions are healed, which can affect availability.   
If the service wants to remain available, it needs to allow the request, which will affect consistency.  
Can't have consistency and availability at the same time.

### CAP theorem

Impossible to get consistency, availability, partition tolerance in distributed system.

Partion tolerance is necessary, so we can't avoid it. AWS chooses availability over consistency, and so do most distributed systems. So we need to know how long inconsistency lasts, and how to resolve inconsistency

### Server side data replication issues

Controlled by three factors:
1. Number of nodes on which data is stored
2. number of nodes from which data will be read from
3. Number of nodes node will be written to.  
If R + W > N, setup will support strong consistency, isnce there will be overlap between read replica and write replica set.   
If R + W = N, then there is a possibility of a disjoint replica set. Clients could have stale time during the time nodes have not reconciled their state.

# Amazon Dynamo

How to support tens of millions of people using tens of thousands of servers located in many data centers around the world?   
Lots of services, and reliability/scalability for the service is dependent on how state is managed.   
Relational databases not scalable enough, too much data ad too many clients. Requirements for new technology is it has to be highly available and built from cheap servers

### Dynamo Contexts

Each service is exposed through a well defined interface, accessible over the network, hosted in infrastructure consistong of tens of thousands of servers located across many data centers. Some services are stateless, some are stateful

### Why did relational databases not work for Amazon?

Required specially trained people, querying capabilities were too simple, and preferred consistency over availability, and data partitioning was inadequate for Amazon.

### Dynamo design requirements

Easy read/writes, no complex operations, single key updates, availability over consistency, and 99.9th percentile of SLA (request speed) limits

### Design considerations

Availability over consistency leads to issues of conflicts (as mentioned like 20x before).   
Dynamo is always writeable, so inconsistencies resolved at read time, and involves services in conflict resolution. Data store can only implement last write wins policy.  
1. Should support adding new nodes
2. Each node should be similar
3. Decentralized data store
4. Possible to use different nodes.   

  
Needs to be fast (so need load balancing), and always writeable.   
Interface is get (which locates object associated with key and returns a list of objects with conflicting versions with a context) and put (which saves the object by determining where the replicas of the objects should be placed based on key. context is saved as metadata with the object)

### Data Partitioning

Uses modified version of consistent hashing.   
Modifications are needed since random position assignments lead to non uniform data and load distribution, and doesn't take into account the different nodes. The modification is multiple key ranges on a node, so each node acts as more than one node.

### Advantages of virtual node

If a node is unavailable, the load handled by the node is evenly dispersed among remaining available nodes. So, if the node goes down, it can be distributed to other nodes, since key range allotment is random.  
You can allocate nodes with more key ranges if they are better nodes

### Replication

Replication factor can be set per instance of dynamo, and keys are stored in at N-1 clockwise successor nodes in the coordinator node for that key. Preference is a list of nodes responsible for storing a key maintained by the coordinator.

### Data Versioning

Basically Causal relations, remember the line stuff we did in OS with event ordering.   
Two events are causally related if one affects the other, and the one affecting happens before the one affected.   
They are not related if they don't affect each other.

### Tracking causality

Done with a vector clock pair of node and counter for each data object, node increments counter when it needs to update data object. If a data object update is handled by different node, then the vector clocks at two nodes will differ, they are not causally related, and are parallel versions.  
If a get request is called, the versions need to be merged.

### Get and Put execution

Invoked over HTTP, two strategies possible
1. Client to load balancer to node, which can be slow and cause latency
2. Client to Node, Client needs to be aware of key space partitioning to find apropriate node for key, low latency.  
  
Client sends request to coordinator, coordinator use preference list and sends request to the top N nodes (N- top N nodes in preference list, R- read replicas or the number of nodes to be contacted before replys to get, w- write replicas, number of nodes to be contacted before replys to put).   
  
Get- coordinator requests all existing versions for data from N highest nodes from preference list, coordinator waits for R responses, if it receives more than R responses, it merges causally dependent versions, and returns all versions that can't be merged.   
   
Put- coordinator generates new vector clock and writes object locally. It sends the new version with the clock to N highest ranked nodes, and if W-1 nodes respond, then the write is successful. W-1 because coordinator wrote object locally.

### Handling network/node failures

Hinted handoff- if a coordinator node failes, other replica nodes in preference list are used to store object, which maintain information about which node was supposed to be the coordinator. On recovery, temporary storage nodes send object to the coordinator.

### Replica Synchronization

Done in background with Merkle trees, which reduces data exchange while checking for inconsistencies. Made of hashes of object keys, parent nodes higher in tree are hashes of children. 

### Ring membership

Nodes can be down for maintenance, no need to rebalance/redistribute keys in this case.   
Some Dynamo nodes play role of a seed node, whose address is defined at a well known location. Each node detects failure of other nodes if there is no response to messages.

# DynamoDB

Fully managed (managed by AWS) NoSQL database    
Features:
1. Define table throughput capacity for each table
2. On demand backup
3. Encryption at rest
4. Auto delete expired items
5. Replicated data
6. Global tables, allows syncing across regions   
   
Names are case sensitive, table, index, attribute names have a length limit   
An attribute can have scalar data, document types, and set types. When creating a table or index, the data type needs to be specified.   
Primary key can only be string. number, binary

### Primary Keys

Primary key needs to be specified when creating a table, two kinds of keys
1. Partition key, composed of one attributes, used with hash function to determine where to store item, unique amongst all primary key
2. Partition and Sort key (range attribute), stored together in sorted order by sorted key value. Many items can have the same primary key value, but different sort key values

### API

1. Control Plane - work with table and streams
2. Data Plane - work with data items in table
3. Streams - work with streams

### Query

Hash/Partition key needs to be specified in query call

### Condition Expressions

used to determine with item should be modified, if true, operation succeeds, otherwise, operation fails.   
: indicates an expression attribute value, acts as a placeholder for the actual value provided later. Use this if you don't know the value until runtime

### Scan

No partition key needs to be specified, reads every item in the entire table and returns all items, filterexpression can be used to limit items return, and is applied only after table is scanned. Each scan returns a page of keys, and to go to the next key, a last evaluate key should be passed in to subsequent scans

### Read Consistency

Tables created in one region is separate from tables in other regions, and table data is replicated in multiple availability zones in a region. When data is written, all replicas are updated eventually. Success sent back when W replicas have been written to. Inconsistency window is 1 or less.    
Supports eventually consistent reads and strongly consistent reads.  
Default value is false (eventually consistent, possible to see stale data), can be set to true for consistent reads, but will not work with network partitions

### Throughput capacity for Reads/Writes

Basically how many read/writes can be done on table/index per second.   
Need to be specified at time of creation.  
One read capacity unit: 1 strongly consistent / 2 eventually consistend read/second for a data item up to 4KB   
One write capacity unit: 1 write per second for data item up to 1KB

DynamoDB max item size is 400KB, so use S3 to store large stuff, will also throttle if read/write goes over the specified capacity throughput

### Throughput capacity management

Autoscaling- define a range for read/write capacity units, define target utilization percentage within that range, request throttling not used, throughput increased/decreased dynamically   
Provisioned throughput- Maximum amount of capacity that application can consume a table/index. Request throttling is done if application exceeds defined capacity.   
Reserved capcity- can be purchased to avoid request throttling

### Good practice

To choose partition keys, choose something that has a large variety of values and no concentration of values.   
For items, use one to many tables instead of large set attributes    
Compress large attribute values, store large attribute values in S3, and break up large attributes across multiple items

### Secondary Indexes

Secondary index is a copy of a table with only selected attributes. Default is secondary index attributes and primary key attributes of the table. A secondary index allows querying the table using an alternate key in addition to using the primary key.  
Global secondary index is an index with a partition and sort key that can be different from those on the table. Local secondary index is an index that has the same partition key as the table, but different sort key

### DynamoDB streams

Captures events on the DynamoDB tables, need to be enabled on a table. Each event is a stream record. If an item is modified, added or deleted, the stream captures the before and after state.

### Authentication and Access Control

DynamoDB only supports IAM policies, does not support resource based policies.   
If you use the root account to create a table, your AWS account is the owner of the resource. If you create an IAM user and grant permissions to create a table, you still own it.   
If you create an IAM role with permissions to create a table, anyone who assumes the role can create a table, but you still own the resource.   
Can defined policies for tables/indexes, and specific actions

### Fine grained access control policies

Using condition definition in IAM policy, which allows uers read/write access to certain items and attributes.  
An example is granting permissions on a table, but restricting access based on certain primary key values, or specific columns"

### IAM Condition Operators

IfExists- if key is present in the context, process the key as specified in the policy. If it isn't evaluate the condition element as true.